<a href="https://colab.research.google.com/github/SRAbhinay/Bharath_Intern_ML_Projects/blob/main/movie_recommendation_system.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import numpy as np
import pandas as pd

In [3]:
ratings_df = pd.read_csv('Dataset.csv')
movies_df = pd.read_csv('Movie_Id_Titles.csv')

In [4]:
print("Ratings Dataset:")
print(ratings_df.head())

print("\nMovies Dataset:")
print(movies_df.head())

Ratings Dataset:
   user_id  item_id  rating  timestamp
0        0       50       5  881250949
1        0      172       5  881250949
2        0      133       1  881250949
3      196      242       3  881250949
4      186      302       3  891717742

Movies Dataset:
   item_id              title
0        1   Toy Story (1995)
1        2   GoldenEye (1995)
2        3  Four Rooms (1995)
3        4  Get Shorty (1995)
4        5     Copycat (1995)


In [11]:
movies_df.isnull().sum(0)

item_id    0
title      0
dtype: int64

In [12]:
ratings_df.isnull().sum(0)

user_id      0
item_id      0
rating       0
timestamp    0
dtype: int64

In [6]:
movie_ratings = pd.merge(ratings_df, movies_df, left_on='item_id', right_on='item_id')

In [7]:
user_movie_ratings = movie_ratings.pivot_table(index='user_id', columns='title', values='rating')


In [9]:
user_movie_ratings = user_movie_ratings.fillna(0)

In [10]:
user_movie_ratings

title,'Til There Was You (1997),1-900 (1994),101 Dalmatians (1996),12 Angry Men (1957),187 (1997),2 Days in the Valley (1996),"20,000 Leagues Under the Sea (1954)",2001: A Space Odyssey (1968),3 Ninjas: High Noon At Mega Mountain (1998),"39 Steps, The (1935)",...,Yankee Zulu (1994),Year of the Horse (1997),You So Crazy (1994),Young Frankenstein (1974),Young Guns (1988),Young Guns II (1990),"Young Poisoner's Handbook, The (1995)",Zeus and Roxanne (1997),unknown,Á köldum klaka (Cold Fever) (1994)
user_id,,,,,,,,,,,,,,,,,,,,,
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,2.0,5.0,0.0,0.0,3.0,4.0,0.0,0.0,...,0.0,0.0,0.0,5.0,3.0,0.0,0.0,0.0,4.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
939,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
940,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
941,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [13]:
print("\nUser-Movie Rating Matrix:")
print(user_movie_ratings.head())


User-Movie Rating Matrix:
title    'Til There Was You (1997)  1-900 (1994)  101 Dalmatians (1996)  \
user_id                                                                   
0                              0.0           0.0                    0.0   
1                              0.0           0.0                    2.0   
2                              0.0           0.0                    0.0   
3                              0.0           0.0                    0.0   
4                              0.0           0.0                    0.0   

title    12 Angry Men (1957)  187 (1997)  2 Days in the Valley (1996)  \
user_id                                                                 
0                        0.0         0.0                          0.0   
1                        5.0         0.0                          0.0   
2                        0.0         0.0                          0.0   
3                        0.0         2.0                          0.0   
4        

In [14]:
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
from scipy.sparse.linalg import svds

In [15]:
train, test = train_test_split(user_movie_ratings, test_size=0.2, random_state=42)

In [16]:
def user_based_collaborative_filtering(user_movie_ratings, user_id, num_recommendations=5):
    user_ratings = user_movie_ratings.loc[user_id].dropna()
    similar_users = user_movie_ratings.corrwith(user_ratings, axis=1).sort_values(ascending=False)
    similar_users = similar_users.dropna()

    user_movie_ratings_reset = user_movie_ratings.reset_index()

    recommendations = pd.DataFrame(similar_users, columns=['correlation'])
    recommendations = recommendations.join(user_movie_ratings_reset.set_index('user_id').mean(axis=1).rename('mean_rating'))
    recommendations = recommendations.sort_values(by=['correlation', 'mean_rating'], ascending=False)
    return recommendations.head(num_recommendations)

In [17]:
user_movie_ratings

title,'Til There Was You (1997),1-900 (1994),101 Dalmatians (1996),12 Angry Men (1957),187 (1997),2 Days in the Valley (1996),"20,000 Leagues Under the Sea (1954)",2001: A Space Odyssey (1968),3 Ninjas: High Noon At Mega Mountain (1998),"39 Steps, The (1935)",...,Yankee Zulu (1994),Year of the Horse (1997),You So Crazy (1994),Young Frankenstein (1974),Young Guns (1988),Young Guns II (1990),"Young Poisoner's Handbook, The (1995)",Zeus and Roxanne (1997),unknown,Á köldum klaka (Cold Fever) (1994)
user_id,,,,,,,,,,,,,,,,,,,,,
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,2.0,5.0,0.0,0.0,3.0,4.0,0.0,0.0,...,0.0,0.0,0.0,5.0,3.0,0.0,0.0,0.0,4.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
939,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
940,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
941,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [18]:
import pandas as pd

def item_based_collaborative_filtering(user_movie_ratings, user_id, num_recommendations=5):
    user_ratings = user_movie_ratings.loc[user_id]

    if user_ratings.dropna().empty:
        return "This user has not rated any movies, so item-based recommendations are not possible."

    similar_movies = user_movie_ratings.corrwith(user_ratings, axis=0).sort_values(ascending=False)
    similar_movies = similar_movies.dropna()

    recommendations = pd.DataFrame(similar_movies, columns=['correlation'])

    mean_ratings = user_movie_ratings.mean(axis=0)

    mean_ratings_df = mean_ratings.reset_index()

    recommendations = recommendations.join(mean_ratings_df, on='title', rsuffix='_mean')
    recommendations.columns = ['title', 'correlation', 'mean_rating']

    recommendations = recommendations.sort_values(by=['correlation', 'mean_rating'], ascending=False)

    return recommendations.head(num_recommendations)



In [19]:
U, sigma, Vt = np.linalg.svd(train)

In [20]:
k = 20
U = U[:, :k]
sigma = np.diag(sigma[:k])
Vt = Vt[:k, :]

In [21]:
predicted_ratings = np.dot(np.dot(U, sigma), Vt)

In [22]:
predicted_ratings_df = pd.DataFrame(predicted_ratings, columns=train.columns)

In [23]:
def svd_recommender(user_id, num_recommendations=5):
    user_ratings = predicted_ratings_df.loc[user_id]
    sorted_user_ratings = user_ratings.sort_values(ascending=False)
    user_rated_movies = train.loc[user_id]
    recommendations = sorted_user_ratings[~sorted_user_ratings.index.isin(user_rated_movies.index)]
    return recommendations.head(num_recommendations)

In [24]:
def calculate_mae(test_data, predicted_data):
    return mean_absolute_error(test_data, predicted_data)

In [26]:
# Example
user_id = 4
user_based_recommendations = user_based_collaborative_filtering(user_movie_ratings, user_id)
print(f"\nUser-Based Collaborative Filtering Recommendations for User {user_id}:")
print(user_based_recommendations)




User-Based Collaborative Filtering Recommendations for User 4:
         correlation  mean_rating
user_id                          
4           1.000000     0.062500
750         0.387308     0.060096
570         0.367479     0.035457
451         0.364961     0.157752
509         0.351105     0.049279


In [27]:
# Example
item_based_recommendations = item_based_collaborative_filtering(user_movie_ratings, user_id)
print(f"\nItem-Based Collaborative Filtering Recommendations for User {user_id}:")
print(item_based_recommendations)


Item-Based Collaborative Filtering Recommendations for User 4:
Empty DataFrame
Columns: [title, correlation, mean_rating]
Index: []


In [28]:
# Example
svd_recommendations = svd_recommender(user_id)
print(f"\nSVD Recommendations for User {user_id}:")
print(svd_recommendations)


SVD Recommendations for User 4:
Series([], Name: 4, dtype: float64)
